In [1]:
import AFQ.tractography as aft
import AFQ.registration as reg
import AFQ.data as afd

import tempfile
import numpy as np
import nibabel as nib
import boto3
import os
import os.path as op
import AFQ.registration as reg

Dataset is already in place. If you want to fetch it again please first remove the folder /root/AFQ_data/templates 


In [2]:
import numpy as np

In [3]:
import dipy.reconst.dti as dti
import dipy.reconst.dki as dki
import dipy.core.gradients as dpg
import dipy.reconst.cross_validation as xval

In [4]:
import nibabel as nib

In [5]:
# Replace with a loop over subjects:
subject = '992774'

In [6]:
def setup_boto():
    boto3.setup_default_session(profile_name='hcp')
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('hcp-openaccess')
    return bucket

In [7]:
def save_wm_mask(subject):
    bucket = setup_boto()
    with tempfile.TemporaryDirectory() as temp_dir:
        dwi_file = op.join(temp_dir, 'data.nii.gz')
        seg_file = op.join(temp_dir, 'aparc+aseg.nii.gz')
        data_files = {}
        data_files[dwi_file] = \
            'HCP/%s/T1w/Diffusion/data.nii.gz' % subject
        data_files[seg_file] = \
            'HCP/%s/T1w/aparc+aseg.nii.gz' % subject
        for k in data_files.keys():
            if not op.exists(k):
                bucket.download_file(data_files[k], k)

        seg_img = nib.load(seg_file)
        dwi_img = nib.load(dwi_file)
        seg_data_orig = seg_img.get_data()
        # Corpus callosum labels:
        cc_mask = ((seg_data_orig==251) | 
                   (seg_data_orig==252) |
                   (seg_data_orig==253) |
                   (seg_data_orig==254) |
                   (seg_data_orig==255))

        # Cerebral white matter in both hemispheres + corpus callosum
        wm_mask = (seg_data_orig==41) | (seg_data_orig==2) | (cc_mask)
        dwi_data = dwi_img.get_data()
        resamp_wm = np.round(reg.resample(wm_mask, dwi_data[..., 0], seg_img.affine, dwi_img.affine)).astype(int)
        wm_file = op.join(temp_dir, 'wm.nii.gz')
        nib.save(nib.Nifti1Image(resamp_wm.astype(int), dwi_img.affine), wm_file) 
        boto3.setup_default_session(profile_name='cirrus')
        s3 = boto3.resource('s3')
        s3.meta.client.upload_file(wm_file, 
                                   'hcp-dki', 
                                   '%s/%s_white_matter_mask.nii.gz'%(subject, subject))

    return (subject, data_files)

In [8]:
#save_wm_mask(subject)

In [9]:
def calc_cod(model, data, mask=None, folds=5):
    pred = xval.kfold_xval(model, data, folds, mask=mask)
    cod = xval.coeff_of_determination(pred, data)
    return cod

In [22]:
def compare_models(subject):
    bucket = setup_boto()
    with tempfile.TemporaryDirectory() as temp_dir:
        dwi_file = op.join(temp_dir, 'data.nii.gz')
        bvec_file = op.join(temp_dir, 'data.bvec')
        bval_file = op.join(temp_dir, 'data.bval')

        data_files = {}
        
        data_files[dwi_file] = \
            'HCP/%s/T1w/Diffusion/data.nii.gz' % subject
        data_files[bvec_file] = \
            'HCP/%s/T1w/Diffusion/bvecs' % subject
        data_files[bval_file] = \
            'HCP/%s/T1w/Diffusion/bvals' % subject
            
        for k in data_files.keys():
            if not op.exists(k):
                bucket.download_file(data_files[k], k)
        dwi_img = nib.load(dwi_file)
        data = dwi_img.get_data()
        gtab = dpg.gradient_table(bval_file, bvec_file, b0_threshold=10)
        boto3.setup_default_session(profile_name='cirrus')
        s3 = boto3.resource('s3')
        for model_object, method in zip([dti.TensorModel, dki.DiffusionKurtosisModel],
                                        ['dti', 'dki']):
            
            print("1")
            model = model_object(gtab)
            print("2")
            cod = calc_cod(model, data) 
            print("3")
            cod_file = op.join(temp_dir, 'cod_%s.nii.gz'%method)
            print("3")
            nib.save(nib.Nifti1Image(cod, dwi_img.affine), cod_file)
            print("4")
            s3.meta.client.upload_file(cod_file, 
                                       'hcp-dki', 
                                       '%s/%s_cod_%s.nii.gz'%(subject, subject, method))

        
        

In [23]:
import time

In [24]:
t1 = time.time()
compare_models(subject)
t2 = time.time()
delta_t = t2 - t1

1
2


MemoryError: 

In [ ]:
print(delta_t)